In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR

In [0]:
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
from google.colab import files

uploaded = files.upload()

Saving Hitter.csv to Hitter.csv


In [0]:
df=pd.read_csv("./Hitter.csv")
df=df.dropna() #eksik değerler siliniyor
dms=pd.get_dummies(df[['League', 'Division', 'NewLeague']]) #kategorik değişkenler dummy e çevrildi.One hot encoding yaklaşımı yapılmış oluyor.
y=df["Salary"]  #bağımlı değişken
X_=df.drop(['Salary','League','Division','NewLeague'], axis=1).astype('float64')
X=pd.concat([X_, dms[['League_N','Division_W','NewLeague_N']]], axis=1) #oluşturulan dummyler ve bir önceki veri seti bir araya getirilerek bağımsız değişkenlerin nihai hali elde edildi.
X_train, X_test, y_train, y_test=train_test_split(X,
                                                  y,
                                                  test_size=0.25,
                                                  random_state=42) #veri seti 25e 75 ayrılmış oldu. test train

In [5]:
!pip install catboost

     |████████████████████████████████| 64.8MB 59kB/s 


In [0]:
from catboost import CatBoostRegressor

In [0]:
catb_model=CatBoostRegressor().fit(X_train,y_train)

In [0]:
y_pred=catb_model.predict(X_test)

In [9]:
np.sqrt(mean_squared_error(y_test, y_pred))

350.2683163098795

**Model Tuning**

In [0]:
catb_params={"iterations":[200,500,1000],
             "learning_rate":[0.01,0.1],
             "depth":[3,6,8]}

In [0]:
catb_model=CatBoostRegressor()

In [12]:
catb_cv_model=GridSearchCV(catb_model, catb_params, cv=5, n_jobs=-1, verbose=2).fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   17.2s


0:	learn: 425.7900818	total: 823us	remaining: 164ms
1:	learn: 404.8723520	total: 1.75ms	remaining: 174ms
2:	learn: 387.4057666	total: 2.56ms	remaining: 168ms
3:	learn: 372.2801584	total: 3.25ms	remaining: 159ms
4:	learn: 358.9204229	total: 3.94ms	remaining: 154ms
5:	learn: 347.0083933	total: 4.64ms	remaining: 150ms
6:	learn: 336.0130818	total: 5.39ms	remaining: 149ms
7:	learn: 324.3923300	total: 6.11ms	remaining: 147ms
8:	learn: 314.8690957	total: 6.78ms	remaining: 144ms
9:	learn: 308.5075563	total: 7.44ms	remaining: 141ms
10:	learn: 298.8587285	total: 8.17ms	remaining: 140ms
11:	learn: 294.7655438	total: 8.9ms	remaining: 139ms
12:	learn: 288.0697862	total: 9.61ms	remaining: 138ms
13:	learn: 282.6697154	total: 10.4ms	remaining: 138ms
14:	learn: 277.6121667	total: 11.1ms	remaining: 136ms
15:	learn: 273.4383979	total: 11.8ms	remaining: 136ms
16:	learn: 269.1556201	total: 12.6ms	remaining: 135ms
17:	learn: 264.8098704	total: 13.3ms	remaining: 134ms
18:	learn: 261.6700768	total: 14ms	remai

[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  2.4min finished


In [13]:
catb_cv_model.best_params_

{'depth': 3, 'iterations': 200, 'learning_rate': 0.1}

In [0]:
catb_tuned=CatBoostRegressor(learning_rate=0.1,
                                    iterations=200,
                                    depth=3).fit(X_train,y_train)

In [16]:
y_pred=catb_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

344.3125832615482